### 使用 pattern mining找出所有的pattern，然后按照最常用的组合进行推荐
按照学生人数为110计算 实际学生为134，但是有些学生的log不符合要求。

In [9]:
from prefixspan import PrefixSpan

db = [
    [0, 1, 2, 3, 4],
    [1, 1, 1, 3, 4],
    [2, 1, 2, 2, 0],
    [1, 1, 1, 2, 2],
]

ps = PrefixSpan(db)

In [1062]:
len(event_c)

3279

In [12]:
print(ps.frequent(2))

[(2, [0]), (4, [1]), (3, [1, 2]), (2, [1, 2, 2]), (2, [1, 3]), (2, [1, 3, 4]), (2, [1, 4]), (2, [1, 1]), (2, [1, 1, 1]), (3, [2]), (2, [2, 2]), (2, [3]), (2, [3, 4]), (2, [4])]


In [334]:
import re
import json
import pandas as pd
# import datetime
import re
from collections import Counter
import numpy as np
import time
import math
import sys
quiz = pd.read_csv(filepath_or_buffer= "C:\\Users\\zhazhang\\Desktop\\GitHub\\quizzResult.csv")
data1 = pd.read_csv(filepath_or_buffer="C:\\Users\\zhazhang\\Desktop\\GitHub\\event_semester1.csv")
exam = pd.read_csv(filepath_or_buffer="C:\\Users\\zhazhang\\Desktop\\GitHub\\exam_result.csv")
data1 = (data1[data1['action'].isin(['Viewed'])])
quiz.score = quiz.score.map(lambda x: x*10)
exam['score'] = exam.score.map(lambda x: (x/18*100))
quiz['action'] = "Submitted"
event = pd.concat([data1[['userId','objectId','action','Timestamp']],quiz[['userId','objectId','action','Timestamp']]])
exam['action']='FinalExam'
exam['Timestamp'] = 1542835584754#1542835584754 # this is the right timestamp of exam
exam['objectId'] = 11209 # I faked an
#event = pd.concat([event, exam[['userId','objectId','action','Timestamp']]])

"""
Here is the treat of event which add the correlated quiz and exam score 
"""
user_list = list(set(event['userId']))
next_score = []
quiz1 = list(set(quiz.objectId))[0]
quiz2 = list(set(quiz.objectId))[1]
exam_time = exam.iloc[0]['Timestamp']
user_quized = []
# print(len(user_list))
for i in user_list:
    if ((len(quiz[quiz.userId==i])) == 2) & (i in list(exam.userId)):
        #quiz=quiz[quiz.userId!=i]
        user_quized.append(i)
#print(len(user_list))
event = event[event.userId.isin(user_quized)]
#print(len(event))
event = event[event.objectId != 23133]
event=event.sort_values(by='Timestamp', ascending=True)
# resourceIndex = {8935: 0, 8936: 1, 316752: 2, 321242: 3, 321240: 4, 321237:5, 11209:6 }
l = list(resourceIndex.keys())
#event1 = event[event["objectId"] == int(l[0])]
#or event["objectId"] == int(l[1]) or event["objectId"] == int(l[2]) or event["objectId"] == int(l[3]) or event["objectId"] == int(l[4]) or event["objectId"] == int(l[5]) or event["objectId"] == int(l[6])or event["objectId"] == int(l[7])or event["objectId"] == int(l[8]) or event["objectId"] == int(l[9])]

# event2 = event[event["objectId"] == int(l[1])]
# event3 = event[event["objectId"] == int(l[2])]
# event4 = event[event["objectId"] == int(l[3])]
# event5 = event[event["objectId"] == int(l[4])]
# event6 = event[event["objectId"] == int(l[5])]
# event7 = event[event["objectId"] == int(l[6])]

# event8 = event[event["objectId"] == int(l[7])]
# event9 = event[event["objectId"] == int(l[8])]
# event10 = event[event["objectId"] == int(l[9])]
#event = pd.concat([event1,event2,event3,event4,event5,event6,event7])#,event8,event9,event10

### 分组问题
---
因为期末考试占比重最大，我们将其乘2.其他俩quiz各占1.


In [1023]:
score_div = {}
for i,u in enumerate(user_list):
    u_score = 0
    u_score+=(sum(quiz[quiz.userId==u].score)+sum(exam[exam.userId==u].score)*2)
    if sum(exam[exam.userId==u].score)!=0:
        score_div[u]=u_score/(len(exam[exam.userId==u].score)*2+len(quiz[quiz.userId==u].score))

In [1024]:
exam[exam.userId=="gonand2u"].score

43    90.277778
42    90.277778
Name: score, dtype: float64

In [1025]:
#总结一下，将三个成绩放到一起看。
quiz_8936 = []
quiz_8996 = []
total_average = []
for i in list(exam.userId):
        user_log = quiz[quiz.userId==i]
        if list(user_log[user_log.objectId==8936].score) != []:
            quiz_8936.append(user_log[user_log.objectId==8936].iloc[0]["score"])
        else:
            quiz_8936.append(0)
        if list(user_log[user_log.objectId==8996].score) != []:
            quiz_8996.append(user_log[user_log.objectId==8996].iloc[0]["score"])
        else:
            quiz_8996.append(0)
        total_average.append(score_div[i])
exam["quiz_score"] = quiz_8996
exam["begin_score"] = quiz_8936
exam["average"] = total_average
exam = exam[exam.begin_score != 0]
exam = exam[exam.score != 0]

In [1026]:
exam.to_csv("exam_withtwoscore.csv")

In [1017]:
len(exam)

104

In [1010]:
print(i)
print(list(user_log[user_log.objectId==8936].score))

wolter6u
[96.4286]


In [999]:
(score_div)

102

In [1003]:
set(quiz.objectId)

{8936, 8996}

In [ ]:
s_good = [] #top 30%
s_normal = []# middle 40%
s_bad = []# next 30%

exam=exam.sort_values(by='score', ascending=False)
for i in range(len(exam)):
    if i < 30:
        # Make sure that this student review on the LMS
        try:
            if event_r[event_r.userId==exam.iloc[i]["userId"]].iloc[0]['objectId']:
                s_good.append(exam.iloc[i]["userId"])
        except IndexError:
            continue
    if i>=30 and i<60:
        try:
            if event_r[event_r.userId==exam.iloc[i]["userId"]].iloc[0]['objectId'] and i not in s_good:
                s_normal.append(exam.iloc[i]["userId"])
        except IndexError:
            continue
    if i>=60 and i<100:
        try:
            if event_r[event_r.userId==exam.iloc[i]["userId"]].iloc[0]['objectId'] and i not in s_good and i not in s_normal:
                s_bad.append(exam.iloc[i]["userId"])
        except IndexError:
            continue

In [353]:
event = event[event.objectId != 23133]
event_c = event[event.Timestamp < 1542835584755]# course
event_r = event[event.Timestamp >=1542835584755]# review
exam=exam.sort_values(by='score', ascending=False)

print(len(event),len(event_r)/len(s_good),len(event_c))

3972 46.2 3279


In [484]:
len(set(exam.userId))

103

In [762]:
s_good = [] #top 30%
s_normal = []# middle 40%
s_bad = []# next 30%

exam=exam.sort_values(by='score', ascending=False)
for i in range(len(exam)):
    if i < 30:
        # Make sure that this student review on the LMS
        try:
            if event_r[event_r.userId==exam.iloc[i]["userId"]].iloc[0]['objectId']:
                s_good.append(exam.iloc[i]["userId"])
        except IndexError:
            continue
    if i>=30 and i<60:
        try:
            if event_r[event_r.userId==exam.iloc[i]["userId"]].iloc[0]['objectId'] and i not in s_good:
                s_normal.append(exam.iloc[i]["userId"])
        except IndexError:
            continue
    if i>=60 and i<100:
        try:
            if event_r[event_r.userId==exam.iloc[i]["userId"]].iloc[0]['objectId'] and i not in s_good and i not in s_normal:
                s_bad.append(exam.iloc[i]["userId"])
        except IndexError:
            continue
        
        
print("student level dividing done")

def get_key(dic, value):
    """
    know the value, get keys
    """
    a = [k for k, v in dic.items() if v == value]
    return a[0]

student level dividing done


Training dataset organisation

In [493]:
l_good = []
l_bad = []
l_normal = []
for user in s_good:
    l_path = list(event_c[event_c.userId==user].objectId)
    l_good.append(l_path)
for user in s_normal:
    l_path = list(event_c[event_c.userId==user].objectId)
    l_normal.append(l_path)
for user in s_bad:
    l_path = list(event_c[event_c.userId==user].objectId)
    l_bad.append(l_path)


In [386]:
ps = PrefixSpan(l_good)
pattern = ps.topk(20, filter=lambda patt, matches: len(patt)>1)
print(pattern)

[(675, [8935, 8995]), (675, [8935, 8996]), (675, [8936, 8935]), (675, [8936, 8935, 8995]), (675, [8936, 8935, 8996]), (675, [8936, 8995]), (675, [8936, 8996]), (613, [8935, 8995, 333191]), (613, [8935, 8996, 333191]), (613, [8935, 333191]), (613, [8936, 8935, 8995, 333191]), (613, [8936, 8935, 8996, 333191]), (613, [8936, 8935, 333191]), (613, [8936, 8995, 333191]), (613, [8936, 8996, 333191]), (613, [8936, 333191]), (613, [8995, 333191]), (613, [8996, 333191]), (611, [8936, 8935, 319715]), (611, [8936, 8996, 8995])]


### recommendation def

In [968]:
#resourcelist = list(set(event.objectId))
resourcelist = [8995, 8996, 325071, 318151, 328647, 316747, 316748, 316749, 316750, 316751, 316752, 325068, 325070, 325069, 325072, 321237, 321238, 321239, 321240, 321241, 321242, 330708, 330711, 319715, 8935, 8936, 32245, 322555]
print(resourcelist)
def recommend(trainingset=l_good, s_group=s_good, student=s_good[0], path_length=9, rl=resourcelist,event=event_r):
    print("student is: ",student)
    # Here we put the influence or this stdent's learning log bigger. x10
    for i in range(30):
        trainingset.append(list(event[event.userId==student].objectId))
    ps = PrefixSpan(trainingset)
    
    pattern = ps.topk(1000, filter=lambda patt, matches: len(patt)>1)# pattern lenth should bigger than 1
    pattern_time = {} #Here stores all pattern with appear times
    
    for i,element in enumerate(pattern):
        l_s = []# store pattern in this element
        s = ""
        for i in range(len(element[1])):
            if i==0:
                s=str(element[1][i])
            else:
                l_s.append(s+","+str(element[1][i]))
                s = str(element[1][i])
        for j in l_s:
            if j in pattern_time.keys():
                pattern_time[j]+=element[0]
            else:
                pattern_time[j]=element[0]
                
    # ordered pattern in list            
    pattern_time = sorted(pattern_time.items(),key = lambda pattern_time:pattern_time[1],reverse=True)
    
    # delete repeat part
    #print(len(pattern_time))
    
    """
    Here is deduplication.
    we can't delete the item of list in for cycle. It will have 'index out of range problem'. 
    So we store the repeat index and delete after
    """ 
    delete_indice = [] 
    for k1 in range(len(pattern_time)):
        starter = pattern_time[k1][0].split(",")[0]
        ender = pattern_time[k1][0].split(",")[1]
        if starter == ender:
            delete_indice.append(k1)
        if pattern_time[k1]==pattern_time[-1]:
            break
            
        for k2 in range(k1+1,len(pattern_time)):
            #print(pattern_time[k2])
            temps_start = pattern_time[k2][0].split(",")[0]
            temps_end = pattern_time[k2][0].split(",")[1]
            if starter == temps_start:
                delete_indice.append(pattern_time[k2])
            if ender == pattern_time[k2][0].split(",")[1]:
                delete_indice.append(pattern_time[k2])    
                
#     for  i in set(delete_indice):
#         if i in pattern_time:
#             pattern_time.remove(i)
#     print("pattern with frequence:",pattern_time)
        
    """
    Here we organise the path from pattern list.
    We should firstly find the head then finish the path.
    """
    element = []
       
    pattern_result = [x[0] for x in pattern_time] # delete pattern show times, keep pattern
    print("unique pattern:",pattern_result)  
    store = []
    for i in range(len(pattern_result)):
        for j in range(len(pattern_result)):
            if i==j:
                continue
            if pattern_result[i].split(",")[0] in pattern_result[j]:
                store.append(pattern_result[i])
#     path =list(set(pattern_result).difference(set(store)))[0] #这里的是删除了不同
#     print("begin_node of path:",path)
    #print(event[event.userId==student]['objectId'])
    try:
        path = str(event[event.userId==student].iloc[0]['objectId'])
    except IndexError:
        path = '8995'
    compt=0
    c_b = 0
    l_change = 2
    repeat_times = 0
    used_pattern = {}
    while compt < path_length-1: # First node has two element, so we should add path_length-2
        c_b+=1
        for i in pattern_result:
#             if i.split(",")[0]==i.split(",")[-1]:# delete repeat in pattern，如 [1,1]
#                 continue
            if i in used_pattern.keys():
                if used_pattern[i]>1:
                    continue
                
            if ',' in path:
                if i.split(",")[0]==path.split(",")[-1]:
                    path+=","+i.split(",")[-1]
                    compt+=1
                    if i in used_pattern.keys():
                        used_pattern[i]+=1
                    else:
                        used_pattern[i]=1
            else:
                if i.split(",")[0]==path:
                    path+=","+i.split(",")[-1]
                    compt+=1
                    used_pattern[i]=1
        if l_change==len(path):
            c_b+=1
        else:
            l_change=len(path)
        if c_b>100:
            path+=","+i
            print("i:",i)
            compt+=2
            if i in used_pattern.keys():
                used_pattern[i]+=1
            else:
                used_pattern[i]=1
    print("path:",path)
    return path 

path = recommend()

[8995, 8996, 325071, 318151, 328647, 316747, 316748, 316749, 316750, 316751, 316752, 325068, 325070, 325069, 325072, 321237, 321238, 321239, 321240, 321241, 321242, 330708, 330711, 319715, 8935, 8936, 32245, 322555]
student is:  wolter6u
unique pattern: ['8936,8935', '8935,8995', '8935,8996', '321237,327938', '327938,333191', '8935,8935', '316752,8935', '327938,327938', '8995,327938', '316747,316752', '316747,8936', '8995,321237', '321237,333191', '8936,8996', '8996,327938', '8996,321237', '8936,8995', '8935,316747', '8935,321237', '8995,333191', '316747,327938', '8995,8995', '316747,8935', '8996,8995', '316747,321237', '316752,8936', '327967,327938', '8996,333191', '8935,327938', '316752,316747', '325068,327938', '316752,8995', '316752,8996', '8995,327967', '8995,327940', '8936,316747', '333191,333191', '8935,333191', '8996,327967', '316747,316749', '316747,316748', '316748,8935', '316749,8935', '316747,333191', '327938,319715', '316747,325068', '8936,321237', '8996,327940', '319715,3

### Calculating metrics and show the result
---
* bad student PM b_pm
* good student PM g_pm
* normal student PM n_pm
* bad student MC b_mc
* good student MC g_mc
* normal student MC n_mc

---
#### Here PM first 

In [623]:
"""
Bad student
"""
n_pm = []
print("normal students:\n")
for i in s_normal:
    a = recommend(l_normal,s_normal, i,6,resourcelist)
    #trainingset=l_normal, s_group=s_normal, student=s_normal[0], path_length=5, rl=resourcelist
    if len(a)>=6:
        g_pm.append(a)
    else:
        print("failed, Unqualified learning data") 

g_pm = []
print("good students:\n")
for i in s_good[:10]:
    #trainingset=l_normal, s_group=s_normal, student=s_normal[0], path_length=5, rl=resourcelist
    a = recommend(l_good,s_good, i,6,resourcelist)
    if len(a)>=6:
        g_pm.append(a)
    else:
        print("failed, Unqualified learning data")
b_pm = []
print("bad students:\n")
for i in s_bad[:10]:
    #trainingset=l_normal, s_group=s_normal, student=s_normal[0], path_length=5, rl=resourcelist
    a = recommend(l_bad,s_bad, i,6,resourcelist)
    if len(a)>=6:
        b_pm.append(a)
    else:
        print("failed, Unqualified learning data")

normal students:

student is:  vignot1u
unique pattern: ['8936,8935', '8996,8995', '8995,8995', '8935,8996', '8935,8995', '8995,333191', '8995,327938', '8936,8996', '316751,8936', '8995,327940', '8936,8995', '327938,327959', '327940,333191', '316750,8936', '327959,333191', '316747,8935', '316747,8936', '316751,8935', '8935,321237', '8996,327938', '8995,327959', '325071,333191', '325072,333191', '327938,333191', '8996,333191', '8995,325072', '8995,321237', '316750,8935', '8995,325071', '8996,327940', '321237,322555', '8995,322555', '316747,316752', '316747,8996', '316752,8935', '316747,316747', '321237,333191', '319715,316747', '8935,322555', '8936,321237', '316747,8995', '8935,327938', '316751,8996', '322555,325071', '322555,325072', '321237,327938', '8935,333191', '8935,8935', '327938,328647', '316747,333191', '8935,316747', '8996,327959', '8935,327940', '316751,8995', '316750,8996', '318151,8935', '8936,322555', '316752,8996', '316752,316747', '316750,8995', '8936,327938', '8996,3250

unique pattern: ['8936,8935', '8996,8995', '8995,8995', '8935,8996', '8935,8995', '8995,333191', '8995,327938', '8936,8996', '316751,8936', '8995,327940', '8936,8995', '327940,333191', '327938,327959', '316747,8936', '316750,8936', '316747,8935', '327959,333191', '316751,8935', '8935,321237', '8995,322555', '8996,327938', '8995,327959', '325071,333191', '325072,333191', '327938,333191', '8996,333191', '316750,8935', '8995,325072', '8995,321237', '8996,327940', '8995,325071', '321237,322555', '316747,316752', '316747,8996', '316747,316747', '316752,8935', '321237,333191', '316747,8995', '8935,322555', '8936,321237', '316751,8996', '8935,327938', '8935,8935', '8935,333191', '327938,328647', '322555,325071', '322555,325072', '321237,327938', '316747,333191', '8935,316747', '8935,327940', '8996,327959', '316751,8995', '319715,316747', '316750,8996', '318151,8935', '8936,322555', '316752,8996', '316752,316747', '8996,322555', '316750,8995', '316748,8936', '316748,316747', '8936,327938', '31

unique pattern: ['8936,8935', '8996,8995', '8995,8995', '8935,8996', '8935,8995', '8995,333191', '8995,327938', '8936,8996', '316751,8936', '8995,327940', '8936,8995', '327938,327959', '327940,333191', '316750,8936', '327959,333191', '316747,8935', '316747,8936', '316751,8935', '8935,321237', '8996,327938', '8995,327959', '325071,333191', '325072,333191', '8996,333191', '327938,333191', '8995,321237', '8995,325072', '316750,8935', '8995,325071', '8996,327940', '321237,322555', '8995,322555', '316747,316752', '316747,8996', '316752,8935', '316747,316747', '321237,333191', '319715,316747', '8935,322555', '316747,8995', '8936,321237', '8935,327938', '316751,8996', '8935,8935', '8935,333191', '327938,328647', '322555,325071', '322555,325072', '321237,327938', '316747,333191', '8935,316747', '8996,327959', '8935,327940', '316751,8995', '316750,8996', '318151,8935', '316752,8996', '8936,322555', '316752,316747', '8936,327938', '316750,8995', '8996,321237', '316752,8995', '8996,325072', '3167

unique pattern: ['8936,8935', '8996,8995', '8995,8995', '8935,8996', '8935,8995', '8995,333191', '8995,327938', '8936,8996', '316751,8936', '8995,327940', '8936,8995', '327938,327959', '327940,333191', '316750,8936', '327959,333191', '316747,8935', '316747,8936', '316751,8935', '8935,321237', '8996,327938', '8995,327959', '325071,333191', '325072,333191', '327938,333191', '8995,325072', '8996,333191', '8995,321237', '316750,8935', '8995,325071', '8996,327940', '321237,322555', '8995,322555', '316747,316752', '316747,8996', '316752,8935', '316747,316747', '321237,333191', '319715,316747', '8935,322555', '8936,321237', '316747,8995', '8935,327938', '316751,8996', '8935,8935', '327938,328647', '322555,325071', '322555,325072', '321237,327938', '8935,333191', '316747,333191', '8935,316747', '8996,327959', '8935,327940', '316751,8995', '316750,8996', '318151,8935', '8936,322555', '316752,8996', '316752,316747', '8936,327938', '316750,8995', '8996,325072', '8996,321237', '316752,8995', '8995

unique pattern: ['8936,8935', '8935,8996', '8935,8995', '321237,327938', '327938,333191', '316752,8935', '327938,327938', '8995,327938', '316747,316752', '316747,8936', '321237,333191', '8996,327938', '8935,8935', '8936,8996', '8996,8995', '8995,321237', '8995,333191', '8936,8995', '333191,333191', '8935,321237', '8935,316747', '8996,321237', '316747,8935', '316747,321237', '316747,327938', '8995,8995', '316752,8936', '8995,327940', '327967,327938', '8996,333191', '316752,316747', '8935,327938', '327940,333191', '8935,319715', '327938,327967', '316752,8995', '8995,327967', '316752,8996', '8996,327940', '319715,321237', '316747,333191', '8935,333191', '8936,316747', '316747,316749', '316749,8935', '327938,319715', '316747,316748', '316748,8935', '319715,316747', '8996,327967', '8936,321237', '316747,8995', '319715,327938', '8995,319715', '325068,333191', '8936,327938', '327940,327938', '327940,319715', '316747,8996', '8995,325068', '8996,325068', '8935,327940', '8936,319715', '316752,32

unique pattern: ['8936,8935', '8935,8996', '8935,8995', '321237,327938', '327938,333191', '327938,327938', '316752,8935', '8995,327938', '8935,8935', '316747,316752', '8995,321237', '316747,8936', '321237,333191', '8936,8996', '8996,327938', '8996,321237', '8936,8995', '8935,316747', '8935,321237', '8995,333191', '316747,327938', '8996,8995', '316747,8935', '316747,321237', '8995,8995', '316752,8936', '327967,327938', '8996,333191', '316752,316747', '8935,327938', '325068,327938', '8995,327967', '316752,8995', '8995,327940', '316752,8996', '8935,319715', '321237,325068', '8936,316747', '8935,333191', '316747,325068', '327938,327967', '8996,327967', '333191,333191', '319715,321237', '316747,316749', '316747,333191', '316747,316748', '316748,8935', '316749,8935', '327938,319715', '8936,321237', '8996,327940', '8996,316747', '316747,8995', '325068,333191', '319715,316747', '8995,325068', '8996,325068', '8936,327938', '8995,316747', '327940,327938', '327940,333191', '8995,319715', '316747,

unique pattern: ['8936,8935', '325068,327938', '316747,321237', '321237,325068', '8935,8996', '327938,321237', '316747,316747', '8935,316747', '8935,8995', '327938,327938', '319715,316747', '321239,325068', '321237,327938', '316747,327938', '8995,327938', '8936,8996', '321237,321238', '321238,321239', '8996,8995', '8996,327938', '8995,321237', '321237,327960', '8936,8995', '8935,321237', '316747,316748', '8936,316747', '8935,319715', '316747,321239', '8996,321237', '316747,325068', '321239,327938', '327938,325068', '8935,8935', '319715,321237', '8935,327938', '321238,325068', '321237,321239', '327938,316747', '321237,333191', '8995,316747', '8996,316747', '8995,328647', '8936,319715', '321237,328647', '8936,321237', '316748,316747', '316748,321237', '328647,327938', '328647,327960', '325068,325069', '325069,327938', '319715,325068', '8995,325068', '8996,325068', '8935,321238', '316747,321238', '316747,327960', '316747,333191', '321238,327938', '327938,333191', '327938,327960', '8936,32

In [644]:
n_pm = []
print("normal students:\n")
for i in s_normal[-10:]:
    a = recommend(l_normal,s_normal, i,10,resourcelist)
    #trainingset=l_normal, s_group=s_normal, student=s_normal[0], path_length=5, rl=resourcelist
    g_pm.append(a)

normal students:

student is:  pignolet3u
unique pattern: ['8936,8935', '8996,8995', '8995,8995', '8935,8996', '8935,8995', '8995,333191', '8995,327938', '8936,8996', '316751,8936', '8995,327940', '8936,8995', '327940,333191', '327938,327959', '316750,8936', '327959,333191', '316747,8935', '316747,8936', '316751,8935', '8935,321237', '8996,327938', '8995,327959', '325071,333191', '325072,333191', '327938,333191', '8996,333191', '8995,325072', '316750,8935', '8995,321237', '8996,327940', '8995,325071', '321237,322555', '316747,316752', '8995,322555', '316747,8996', '316752,8935', '316747,316747', '321237,333191', '319715,316747', '8935,322555', '8936,321237', '316747,8995', '8935,327938', '316751,8996', '8935,8935', '322555,325071', '322555,325072', '321237,327938', '8935,333191', '327938,328647', '316747,333191', '8935,316747', '8996,327959', '8935,327940', '316751,8995', '316750,8996', '318151,8935', '316752,8996', '8936,322555', '316752,316747', '316750,8995', '8936,327938', '8996,32

unique pattern: ['8936,8935', '8996,8995', '8995,8995', '8935,8996', '8935,8995', '8995,333191', '8995,327938', '8936,8996', '316751,8936', '8995,327940', '8936,8995', '327940,333191', '327938,327959', '316750,8936', '327959,333191', '316747,8935', '316747,8936', '316751,8935', '8935,321237', '8996,327938', '8995,327959', '8996,333191', '325071,333191', '325072,333191', '327938,333191', '8995,325072', '8995,321237', '316750,8935', '8996,327940', '8995,325071', '321237,322555', '316747,316752', '316747,8996', '316752,8935', '316747,316747', '8995,322555', '321237,333191', '319715,316747', '316747,8995', '8935,322555', '8936,321237', '8935,327938', '316751,8996', '8935,8935', '321237,327938', '8935,333191', '322555,325071', '322555,325072', '327938,328647', '316747,333191', '8935,316747', '8996,327959', '8935,327940', '316751,8995', '316750,8996', '318151,8935', '316752,8996', '316752,316747', '8936,322555', '316750,8995', '316752,8995', '8936,327938', '8996,325072', '8996,321237', '3167

unique pattern: ['8936,8935', '8996,8995', '8995,8995', '8935,8996', '8935,8995', '8995,333191', '8995,327938', '8936,8996', '316751,8936', '8995,327940', '8936,8995', '327940,333191', '327938,327959', '316750,8936', '327959,333191', '316747,8935', '316747,8936', '316751,8935', '8935,321237', '8996,327938', '8995,327959', '325071,333191', '325072,333191', '327938,333191', '8996,333191', '8995,325072', '316750,8935', '8995,321237', '8996,327940', '8995,325071', '321237,322555', '316747,316752', '316747,8996', '8995,322555', '316752,8935', '316747,316747', '321237,333191', '319715,316747', '8935,322555', '8936,321237', '316747,8995', '8935,327938', '316751,8996', '321237,327938', '8935,8935', '322555,325071', '322555,325072', '8935,333191', '327938,328647', '316747,333191', '8935,316747', '8996,327959', '8935,327940', '316751,8995', '316750,8996', '318151,8935', '316752,8996', '8936,322555', '316752,316747', '316750,8995', '8936,327938', '8996,325072', '316752,8995', '8996,321237', '3212

In [643]:
n_pm

[]

In [628]:
b_pm

['316750,8996,319715,316747,327938,325068,325069,327938,333191,8996,319715',
 '316750,8996,319715,316747,327938,325068,325069,327938,333191,8996,319715',
 '316750,316747,8996,327938,325068,325069,327938,333191,316747,8996',
 '321240,316747,8996,8995,321237,327960,316747,8996',
 '316750,316747,8996,8995,321237,327960,316747,8996',
 '321240,316747,8996,8995,328647,327938,327960,316747,8996',
 '321240,8936,327960,8936,327960,8936,327960',
 '319715,316747,327938,325068,325069,327938,327960',
 '321240,8996,319715,316747,327938,325068,325069,327938,327960,8996,319715',
 '316750,8996,319715,316747,327938,325068,325069,327938,333191,8996,319715']

### Markov chain method

In [671]:
def transition_diff(resourceIndex, df_total):
    print("calculating the transition matrix from all the records")
    n = len(resourceIndex)  # number of states
    M = np.zeros((n, n))  # Matrix for all the
    UserID = []  # 存储处理的用户的顺序。
    actionspace = []
    for i in set(df_total['userId']):
#         print("treating user :" + i)
        i_user = df_total[df_total['userId'].isin([i])]
        i_user = i_user.sort_values(by='Timestamp')
        UserID.append(i_user)
        listResource = []
        for j in i_user.objectId:
            listResource.append(resourceIndex[j])
        actionspace.append(listResource)  # 将进行过的每个用户的学习过程全部记录下来
        for (i, j) in zip(listResource, listResource[1:]):
            M[i][j] += 1

    # print(sum(M))
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f / s for f in row]
    return M

def get_key(dic, value):
    """
    know the value, get keys
    """
    a = [k for k, v in dic.items() if v == value]
    return a[0]

# 预测状态的马尔可夫模型的函数
state = [i for i in range(len(set(event["objectId"])))]
resourceIndex = {}
for i,e in  enumerate(set(event["objectId"])):
    resourceIndex[e]=i
print(resourceIndex)
def study_forecast(resource=resourceIndex, states=state,student=s_good[0],trainingset=l_good,
                   s_group=s_good, path_length=6,event1=event_c,event2=event_r):
    # define
#     print(event1[event1.userId==student])
    #M_s = transition_diff(resource, event2[event2.userId==student])
    M_s = transition_diff(resource, event2[event2.userId==student])
    print('len M_s:',len(M_s))
    M = transition_diff(resource, event1)
    print('len M',len(M))
    T = [[] for x in M_s]
    for line in range(len(M_s)):
        if sum(M_s[line])>0:
            T[line] = 0.1*M[line]+0.9*M_s[line] 
        else:
            T[line] = M[line]
    # chose initial state
    resourceNow = resourceIndex[event2[event2.userId==student].iloc[0]['objectId']]
    print("Start state: " + str(resourceNow))
    # 应该记录选择的状态序列。这里现在只有初始状态。
    activityList = [resourceNow]
    i = 0
    # 计算 activityList 的概率
    prob = 1
    while i != path_length:
        change = np.random.choice(list(states), replace=True, p=T[resourceNow])
        activityList.append(change)
        prob = prob*T[resourceNow][change]
        resourceNow = change
        i += 1
    print("Possible states: " + str(activityList))
    #print("End state after "+ str(resource) + " resource: " + str(resourceNow))
    print("Probability of the possible sequence of states: " + str(prob))
    return str(activityList)
#study_forecast(resourceIndex,state,s_good[0],l_good,s_good,9,event_c,event_r)

{327938: 0, 327940: 1, 327941: 2, 333190: 3, 333191: 4, 327959: 5, 327960: 6, 327966: 7, 327967: 8, 8995: 9, 8996: 10, 325071: 11, 318151: 12, 328647: 13, 316747: 14, 316748: 15, 316749: 16, 316750: 17, 316751: 18, 316752: 19, 325068: 20, 325070: 21, 325069: 22, 325072: 23, 321237: 24, 321238: 25, 321239: 26, 321240: 27, 321241: 28, 321242: 29, 330708: 30, 330711: 31, 319715: 32, 8935: 33, 8936: 34, 32245: 35, 322555: 36}


# Markov 部分代码结果

In [614]:
n_mc = []
print("normal students:\n")
for i in s_normal[:10]:
    try:
        a = study_forecast(resourceIndex,state,i,l_normal,s_normal,5,event_c,event_r)
        n_mc.append(a)
    except ValueError:
        continue

g_mc = []
print("good students:\n")
for i in s_good[:10]:
    #trainingset=l_normal, s_group=s_normal, student=s_normal[0], path_length=5, rl=resourcelist
    try:
        a = study_forecast(resourceIndex,state,i,l_good,s_good,5,event_c,event_r)
        g_mc.append(a)
    except ValueError:
        continue
    

b_mc = []
print("bad students:\n")
for i in s_bad[3:13]:
    #trainingset=l_normal, s_group=s_normal, student=s_normal[0], path_length=5, rl=resourcelist
    try:
        a = study_forecast(resourceIndex,state,i,l_bad,s_bad,5,event_c,event_r)
        b_mc.append(a)
    except ValueError:
        continue
       


normal students:

calculating the transition matrix from all the records
calculating the transition matrix from all the records
Start state: 4
Possible states: [4, 3, 4, 3, 32, 14]
Probability of the possible sequence of states: 0.04517765318112977
calculating the transition matrix from all the records
calculating the transition matrix from all the records
Start state: 17
calculating the transition matrix from all the records
calculating the transition matrix from all the records
Start state: 17
Possible states: [17, 18, 17, 18, 27, 27]
Probability of the possible sequence of states: 0.059595184765800766
calculating the transition matrix from all the records
calculating the transition matrix from all the records
Start state: 17
calculating the transition matrix from all the records
calculating the transition matrix from all the records
Start state: 17
calculating the transition matrix from all the records
calculating the transition matrix from all the records
Start state: 4
calculating

In [637]:
g_pm

['327938,333191,333191,333191,319715,325068',
 '316750,319715,325068,327938,319715,316747,8996,319715,8935,330708,319715,325068',
 '316751,8995,327938,327967,327938,327967',
 '333190,319715,319715,321237,330708,319715,319715',
 '327941,8936,321239,8936,321239,8936,321239',
 '316750,319715,325068,327938,319715,316747,8996,316747,316747,319715,8935,330708,319715,325068',
 '316750,319715,325068,327938,327967,319715,325068',
 '327938,333191,333191,333191,319715,325068',
 '316751,8995,327938,319715,316747,8996,319715,8935,330708',
 '316747,316752,8936,316747,325068,333191',
 '333191,319715,327959,333191,319715,327959,333191,319715,327959,319715,327959',
 '316750,8936,321237,327938,327960,327938,327960,327938,327960,327938,327960',
 '316750,8936,321237,327938,327960,327938,327960,327938,327960,327938,327960',
 '316750,8936,321237,327938,327959,333191,318151,333191,318151,333191',
 '316750,8936,321237,327938,327959,333191,316751,327959,333191,316751,327959',
 '333191,316751,327959,333191,3167

# 方法一 计算相似度 + 方法二的ED

In [640]:
import numpy as np
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import editdistance
import copy

l =['316750,8936,327960,8936,327960,8936,327960','321240,8936,327960,8936,327960,8936,327960', '321240,8936,327960,8936,327960,8936,327960', '319715,316747,327938,325068,325069,327938,327960', 
'321240,8996,319715,316747,327938,325068,325069,327938,327960,8996,319715', '316750,8996,319715,316747,327938,325068,325069,327938,333191,8996,319715']

#"17,18,15,16,14,13,2,1,23,14".split(",")
for i,e in enumerate(l):
    x = e.split(",")
    x = [resourceIndex[int(a)] for a in x]
    l[i]= x
c_good = []#这个是用来对比的标准
#转换成list

for user in s_good:
    l_path = list(event_r[event_r.userId==user].objectId)
    for r in range(len(l_path)):
        l_path[r] = resourceIndex[l_path[r]]
    c_good.append(l_path)


distance = [x for x in range(len(l))]

for i,e in enumerate(l) :
    for i_c,e_c in enumerate(c_good):
        #d2,p = fastdtw(e, e_c, dist=euclidean)
        e_copy=copy.deepcopy(e_c)
        for j in e_c:
            if j not in e:
                e_copy.remove(j)
        d = editdistance.eval(str(e), str(e_copy))
        if d<=min_v:
            min_v = d
            distance[i]=[e,e_c,e_copy,d]

# 方法二的DTWD
* 先找每个student对应的gss中最近的event_c中的学习路径
* 然后计算推荐路径对应的已学习路径的距离.平均距离

In [814]:
s_bad # bad student lists
min_list = []
for i,e in enumerate(s_bad): # Here we search the pair of mentor student and target learner.
    min_dis = 1000
    for j,k in enumerate(s_good):
        lp_bad = str(list(event_c[event_c.userId==e].objectId))
        lp_good = str(list(event_c[event_c.userId==k].objectId))
        distance = editdistance.eval(lp_good, lp_bad)
        if min_dis > distance:
            if len(min_list)>=i+1:
                min_list[i] = [e,k,distance]
                min_dis = distance
            else:
                min_list.append([e,k,distance])
                min_dis = distance

In [815]:
print(min_list)
print(lp_good)

[['allal11u', 'signer1u', 155], ['lauren329u', 'oudart4u', 57], ['schwab17u', 'mogenot5u', 80], ['maige1u', 'oudart4u', 45], ['allal11u', 'signer1u', 155], ['mangeot38u', 'ulmann2u', 174], ['laghouat8u', 'marais8u', 206], ['schweize3u', 'wolter6u', 334], ['maige1u', 'oudart4u', 45], ['saran2u', 'marais8u', 204], ['lods3u', 'grebert5u', 115], ['laghouat8u', 'marais8u', 206], ['saran2u', 'marais8u', 204], ['humber245u', 'mekhilef3u', 283], ['mangeot38u', 'ulmann2u', 174], ['humber245u', 'mekhilef3u', 283], ['lods3u', 'grebert5u', 115], ['ladli2u', 'braux4u', 147], ['martin711u', 'marais8u', 279]]
[8936, 8935, 8935, 8996, 8995, 8995, 327938, 321238, 321239, 325069, 327938, 327959, 333191]


In [1031]:
s_normal # bad student lists
min_list = []
for i,e in enumerate(s_good): # Here we search the pair of mentor student and target learner.
    min_dis = 1000
    for j,k in enumerate(s_good):
        if e==k:
            continue
        lp_normal = str(list(event_c[event_c.userId==e].objectId))
        lp_good = str(list(event_c[event_c.userId==k].objectId))
        distance = editdistance.eval(lp_good, lp_normal)
        if min_dis > distance:
            if len(min_list)>=i+1:
                min_list[i] = [e,k,distance]
                min_dis = distance
            else:
                min_list.append([e,k,distance])
                min_dis = distance

In [1033]:
['braux4u', 'poirot152u', 88],
['poirot152u', 'oudart4u', 82],
['mogenot5u', 'oudart4u', 68],
['grebert5u', 'braux4u', 94],
a = ['braux4u','poirot152u','mogenot5u','grebert5u']
b = ['poirot152u','oudart4u','oudart4u','braux4u']

In [1038]:
# MC in l_a
l_2 = []
for i in range(len(a)):
    l_a = str(list(event_c[event_c.userId==a[i]].objectId)[:10])
    l_b = study_forecast(resourceIndex,state,a[i],l_good,s_good,10,event_c,event_r)
    l_2.append(editdistance.eval(l_a, l_b))
print(l_2)
l_1 = []
for i in range(len(a)):
    l_a = str(list(event_c[event_c.userId==a[i]].objectId)[:10])
    l_b = study_forecast(resourceIndex,state,a[i],l_good,s_good,10,event_c,event_r)
    l_1.append(editdistance.eval(l_a, l_b))
print(l_1)

calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 17
Possible states: [17, 27, 1, 30, 30, 3, 3, 15, 32, 14, 32]
Probability of the possible sequence of states: 0.011588916290882913
calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 0
Possible states: [0, 3, 18, 27, 23, 7, 32, 15, 36, 14, 15]
Probability of the possible sequence of states: 3.219624008813108e-12
calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 3
Possible states: [3, 4, 32, 14, 15, 16, 17, 18, 17, 18, 19]
Probability of the possible sequence of states: 0.005070502941447334
calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 18
Possible states: [18, 17, 18

###### Here we find that "maige1u" and "lauren329u" share a same teacher user:'oudart4u'

* We first calculate the MC
* Then PM

In [696]:
r_b = study_forecast(resource=resourceIndex, states=state,student="maige1u",trainingset=l_bad,
                   s_group=s_bad, path_length=17,event1=event_c,event2=event_r)

calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 27
Possible states: [27, 2, 2, 2, 2, 11, 23]
Probability of the possible sequence of states: 0.05619820963404902


In [687]:
list(resourceIndex.keys())[0]

327938

In [697]:
r_b = [27, 2, 2, 2, 2, 11, 23]

In [698]:
for i in range(len(r_b)):
    r_b[i] = list(resourceIndex.keys())[int(r_b[i])]

In [699]:
r_b

[321240, 327941, 327941, 327941, 327941, 325071, 325072]

In [702]:
r_b1 = recommend(l_bad,s_bad, 'maige1u',6,resourcelist)

student is:  maige1u
unique pattern: ['325068,327938', '8936,8935', '316747,321237', '321237,325068', '316747,316747', '321239,325068', '327938,321237', '327938,327938', '319715,316747', '8935,8996', '8935,316747', '321237,327938', '316747,327938', '321237,321238', '321238,321239', '8935,8995', '316747,316748', '8936,316747', '8995,327938', '316747,321239', '321239,327938', '327938,325068', '316748,321237', '8935,321237', '8996,327938', '8995,321237', '316747,325068', '8935,319715', '8936,8996', '321238,325068', '321237,321239', '8996,321237', '316748,316747', '8936,8995', '319715,321237', '8996,8995', '325068,325069', '8936,319715', '321237,333191', '325069,327938', '8936,321237', '319715,325068', '8995,325068', '8996,325068', '321237,327960', '8935,327938', '8935,8935', '321238,327938', '8995,316747', '8996,316747', '8935,321238', '316748,321238', '316747,321238', '316747,333191', '327938,333191', '319715,327938', '8995,333191', '316748,321239', '8936,327938', '316748,327938', '32506

In [703]:
r_b1 = recommend(l_bad,s_bad, 'lauren329u',6,resourcelist)

student is:  lauren329u
unique pattern: ['325068,327938', '8936,8935', '316747,321237', '321237,325068', '316747,316747', '321239,325068', '327938,321237', '327938,327938', '319715,316747', '8935,8996', '8935,316747', '321237,327938', '316747,327938', '321237,321238', '321238,321239', '8935,8995', '316747,316748', '8936,316747', '8995,327938', '316747,321239', '321239,327938', '327938,325068', '316748,321237', '8935,321237', '8996,327938', '8995,321237', '316747,325068', '8935,319715', '8936,8996', '321238,325068', '321237,321239', '8996,321237', '316748,316747', '8936,8995', '319715,321237', '8996,8995', '325068,325069', '8936,319715', '321237,333191', '325069,327938', '8936,321237', '319715,325068', '8995,325068', '8996,325068', '321237,327960', '8935,327938', '8935,8935', '321238,327938', '8995,316747', '8996,316747', '8935,321238', '316748,321238', '316747,321238', '316747,333191', '327938,333191', '319715,327938', '8995,333191', '316748,321239', '8936,327938', '316748,327938', '32

## 第二种方法的中等学生

In [1066]:
s_normal # bad student lists
min_list = []
for i,e in enumerate(s_bad): # Here we search the pair of mentor student and target learner.
    min_dis = 1000
    for j,k in enumerate(s_good):
        lp_normal = str(list(event_c[event_c.userId==e].objectId))
        lp_good = str(list(event_c[event_c.userId==k].objectId))
        distance = editdistance.eval(lp_good, lp_normal)
        if min_dis > distance:
            if len(min_list)>=i+1:
                min_list[i] = [e,k,distance]
                min_dis = distance
            else:
                min_list.append([e,k,distance])
                min_dis = distance

In [1067]:
min_list

[['allal11u', 'signer1u', 155],
 ['lauren329u', 'oudart4u', 57],
 ['schwab17u', 'mogenot5u', 80],
 ['maige1u', 'oudart4u', 45],
 ['allal11u', 'signer1u', 155],
 ['mangeot38u', 'ulmann2u', 174],
 ['laghouat8u', 'marais8u', 206],
 ['schweize3u', 'wolter6u', 334],
 ['maige1u', 'oudart4u', 45],
 ['saran2u', 'marais8u', 204],
 ['lods3u', 'grebert5u', 115],
 ['laghouat8u', 'marais8u', 206],
 ['saran2u', 'marais8u', 204],
 ['humber245u', 'mekhilef3u', 283],
 ['mangeot38u', 'ulmann2u', 174],
 ['humber245u', 'mekhilef3u', 283],
 ['lods3u', 'grebert5u', 115],
 ['ladli2u', 'braux4u', 147],
 ['martin711u', 'marais8u', 279]]

In [ ]:
'courtin11u', "benkira12u"

In [711]:
r_b = study_forecast(resource=resourceIndex, states=state,student="courtin11u",trainingset=l_normal,
                   s_group=s_normal, path_length=17,event1=event_c,event2=event_r)

calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 17
Possible states: [17, 18, 33, 34, 33, 33, 15, 17, 18, 27, 28, 31, 27, 28, 1, 0, 5, 6]
Probability of the possible sequence of states: 1.8206054052597195e-11


In [712]:
r_b = [17, 18, 33, 34, 33, 33, 15, 17, 18, 27, 28, 31, 27, 28, 1, 0, 5, 6]
for i in range(len(r_b)):
    r_b[i] = list(resourceIndex.keys())[int(r_b[i])]
print(r_b)

[316750, 316751, 8935, 8936, 8935, 8935, 316748, 316750, 316751, 321240, 321241, 330711, 321240, 321241, 327940, 327938, 327959, 327960]


In [714]:
r_b = study_forecast(resource=resourceIndex, states=state,student="benkira12u",trainingset=l_bad,
                   s_group=s_bad, path_length=17,event1=event_c,event2=event_r)

calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 17
Possible states: [17, 18, 27, 28, 11, 23, 17, 18, 27, 28, 11, 23, 17, 18, 27, 28, 4, 18]
Probability of the possible sequence of states: 0.003244259487558348


In [715]:
r_b = [17, 18, 27, 28, 11, 23, 17, 18, 27, 28, 11, 23, 17, 18, 27, 28, 4, 18]
for i in range(len(r_b)):
    r_b[i] = list(resourceIndex.keys())[int(r_b[i])]
print(r_b)

[316750, 316751, 321240, 321241, 325071, 325072, 316750, 316751, 321240, 321241, 325071, 325072, 316750, 316751, 321240, 321241, 333191, 316751]


In [713]:
r_b1 = recommend(l_normal,s_normal, 'courtin11u',17,resourcelist)

student is:  courtin11u
unique pattern: ['8936,8935', '8996,8995', '8995,8995', '8935,8996', '8935,8995', '8995,333191', '8995,327938', '8936,8996', '316751,8936', '8995,327940', '8936,8995', '327938,327959', '327940,333191', '316750,8936', '327959,333191', '316747,8935', '316747,8936', '316751,8935', '8935,321237', '8996,327938', '8995,327959', '325071,333191', '325072,333191', '327938,333191', '8996,333191', '8995,325072', '316750,8935', '8995,321237', '8996,327940', '8995,325071', '321237,322555', '316747,316752', '316747,8996', '8995,322555', '316752,8935', '316747,316747', '321237,333191', '8935,322555', '8936,321237', '316747,8995', '319715,316747', '8935,327938', '316751,8996', '327938,328647', '321237,327938', '8935,8935', '322555,325071', '322555,325072', '8935,333191', '316747,333191', '8935,316747', '8996,327959', '8935,327940', '316751,8995', '316750,8996', '318151,8935', '316752,8996', '8936,322555', '316752,316747', '8936,327938', '316750,8995', '8996,325072', '316752,899

In [716]:
r_b1 = recommend(l_normal,s_normal, 'benkira12u',17,resourcelist)

student is:  benkira12u
unique pattern: ['8936,8935', '8996,8995', '8995,8995', '8935,8996', '8935,8995', '8995,333191', '8995,327938', '8936,8996', '316751,8936', '8995,327940', '8936,8995', '327938,327959', '327940,333191', '316750,8936', '327959,333191', '316747,8935', '316747,8936', '316751,8935', '8935,321237', '8996,327938', '8995,327959', '325071,333191', '325072,333191', '327938,333191', '8995,325072', '8996,333191', '316750,8935', '8995,321237', '8996,327940', '8995,325071', '321237,322555', '316747,8996', '316747,316752', '316752,8935', '8995,322555', '316747,316747', '321237,333191', '319715,316747', '8935,322555', '8936,321237', '316747,8995', '8935,327938', '316751,8996', '8935,8935', '327938,328647', '321237,327938', '322555,325071', '322555,325072', '8935,333191', '316747,333191', '8935,316747', '8996,327959', '8935,327940', '316751,8995', '316750,8996', '318151,8935', '316752,8996', '316752,316747', '8936,322555', '8936,327938', '316750,8995', '8996,325072', '316752,899

In [726]:
(len(list(event_c[event_c.userId=='courtin11u'].objectId)))

17

In [642]:
distance2 = [x for x in range(len(l))]
min_v = 10000
for i,e in enumerate(l) :
    for i_c,e_c in enumerate(c_good):
        
        e_copy=copy.deepcopy(e_c)
        for j in e_c:
            if j not in e:
                e_copy.remove(j)
        d2,p = fastdtw(e, e_c, dist=euclidean)
        if d2<=min_v:
            min_v = d2
            distance[i]=[e,e_c,e_copy,d]

# 方法三
计算所（平均有的GSS的推荐与实际的距离-平均计算BSS的推荐与实际距离）的值
* PM
* MC

In [736]:
# Here is function of distance.
def cal_dis(l1,l2):
    l1=l1.split(",")
    l1 = [resourceIndex[int(x)] for x in l1]
    l2 = [resourceIndex[x] for x in l2]
    return editdistance.eval(str(l1),str(l2))

In [744]:
normalisation = (10+20*2)/39

In [752]:
print(len(s_good))

15


In [753]:
print(len(s_bad))

19


In [754]:
print(len(s_normal))

13


In [756]:
good_len = 0
for i,s in enumerate(s_good):
    good_len+=len(list(event_r[event_r.userId==s].objectId))
print(good_len/len(s_good))

17.533333333333335


In [757]:
normal_len = 0
for i,s in enumerate(s_normal):
    normal_len+=len(list(event_r[event_r.userId==s].objectId))
print(normal_len/len(s_normal))

13.461538461538462


In [758]:
bad_len = 0
for i,s in enumerate(s_bad):
    bad_len+=len(list(event_r[event_r.userId==s].objectId))
print(bad_len/len(s_bad))

13.473684210526315


In [760]:
23.4-15.13

8.269999999999998

In [737]:
s = 0
for i,student in enumerate(s_good):
    l2 = list(event_r[event_r.userId==student].objectId)
    len_r = len(l2)
    l1 = recommend(trainingset=l_good, s_group=s_good, student=student, path_length=len_r, rl=resourcelist,event=event_r)
    s += cal_dis(l1,l2)
print(s/len(s_good)/normalisation)

#32.13

student is:  wolter6u
unique pattern: ['8936,8935', '8935,8996', '8935,8995', '321237,327938', '327938,333191', '327938,327938', '316752,8935', '8995,327938', '8935,8935', '316747,316752', '8995,321237', '316747,8936', '321237,333191', '8996,321237', '8936,8996', '8996,327938', '8936,8995', '8935,316747', '8935,321237', '8995,333191', '316747,327938', '8996,8995', '316747,8935', '316747,321237', '8995,8995', '316752,8936', '327967,327938', '8996,333191', '316752,316747', '325068,327938', '8935,327938', '8995,327967', '316752,8995', '8995,327940', '316752,8996', '321237,325068', '8935,319715', '8936,316747', '316747,325068', '327938,327967', '8935,333191', '8996,327967', '333191,333191', '319715,321237', '316747,316749', '316747,316748', '316748,8935', '316749,8935', '327938,319715', '316747,333191', '8936,321237', '8996,327940', '8996,316747', '316747,8995', '325068,333191', '319715,316747', '8995,325068', '8996,325068', '8995,316747', '327940,327938', '327940,333191', '8936,327938', '

unique pattern: ['8936,8935', '8935,8996', '8935,8995', '321237,327938', '327938,333191', '316752,8935', '327938,327938', '8995,327938', '8935,8935', '316747,316752', '316747,8936', '321237,333191', '8995,321237', '8996,327938', '8935,316747', '8936,8996', '8996,321237', '8935,321237', '8995,333191', '8936,8995', '316747,327938', '8996,8995', '316747,8935', '316747,321237', '327967,327938', '8995,8995', '316752,8936', '8996,333191', '316752,316747', '8935,327938', '8935,319715', '8995,327940', '8995,327967', '327938,327967', '316752,8995', '316752,8996', '319715,321237', '8936,316747', '333191,333191', '8935,333191', '325068,327938', '8996,327940', '8996,327967', '316747,316749', '316749,8935', '327938,319715', '316747,333191', '316747,316748', '316748,8935', '319715,316747', '8936,321237', '316747,8995', '319715,327938', '325068,333191', '327940,333191', '8995,319715', '327940,327938', '327940,319715', '8995,325068', '8996,325068', '8936,327938', '316747,8996', '8936,319715', '8935,32

unique pattern: ['8936,8935', '8935,8996', '8935,8995', '321237,327938', '327938,333191', '327938,327938', '316752,8935', '8995,327938', '8935,8935', '316747,316752', '8995,321237', '316747,8936', '321237,333191', '8936,8996', '8996,321237', '8996,327938', '8936,8995', '8935,316747', '8935,321237', '8995,333191', '316747,327938', '8996,8995', '316747,8935', '316747,321237', '8995,8995', '316752,8936', '327967,327938', '8996,333191', '316752,316747', '325068,327938', '8935,327938', '8995,327967', '316752,8995', '8995,327940', '316752,8996', '321237,325068', '8935,319715', '8936,316747', '316747,325068', '327938,327967', '8935,333191', '8996,327967', '333191,333191', '319715,321237', '316747,316749', '316747,316748', '316748,8935', '316749,8935', '327938,319715', '316747,333191', '8936,321237', '8996,327940', '8996,316747', '316747,8995', '325068,333191', '319715,316747', '8995,325068', '8996,325068', '8995,316747', '327940,327938', '327940,333191', '8936,327938', '8995,319715', '316747,

unique pattern: ['8936,8935', '8935,8995', '8935,8996', '321237,327938', '327938,333191', '8935,8935', '316752,8935', '327938,327938', '8995,327938', '316747,316752', '316747,8936', '8995,321237', '321237,333191', '8936,8996', '8996,327938', '8996,321237', '8936,8995', '8935,316747', '8935,321237', '8995,333191', '316747,327938', '8995,8995', '316747,8935', '8996,8995', '316747,321237', '316752,8936', '327967,327938', '8996,333191', '8935,327938', '316752,316747', '325068,327938', '316752,8995', '316752,8996', '8995,327967', '8995,327940', '8936,316747', '8935,333191', '333191,333191', '8996,327967', '316747,316749', '316747,316748', '316748,8935', '316749,8935', '316747,333191', '327938,319715', '316747,325068', '8936,321237', '8996,327940', '319715,321237', '8935,319715', '316747,8995', '321237,325068', '327938,327967', '325068,333191', '8995,325068', '8996,325068', '8936,327938', '8995,316747', '8996,316747', '327940,327938', '327940,333191', '316747,8996', '8935,327967', '8995,3197

In [739]:
sum_normal = 0
for i,student in enumerate(s_normal):
    l2 = list(event_r[event_r.userId==student].objectId)
    len_r = len(l2)
    l1 = recommend(trainingset=l_normal, s_group=s_normal, student=student, path_length=len_r, rl=resourcelist,event=event_r)
    sum_normal += cal_dis(l1,l2)
print(sum_normal/len(s_good))
#19.4

student is:  vignot1u
unique pattern: ['8936,8935', '8996,8995', '8995,8995', '8935,8996', '8935,8995', '8995,333191', '8995,327938', '8936,8996', '316751,8936', '8995,327940', '8936,8995', '327938,327959', '327940,333191', '316750,8936', '327959,333191', '316747,8935', '316747,8936', '316751,8935', '8935,321237', '8996,327938', '8995,327959', '325071,333191', '325072,333191', '327938,333191', '8995,325072', '8996,333191', '316750,8935', '8995,321237', '8996,327940', '8995,325071', '321237,322555', '316747,316752', '316747,8996', '316752,8935', '8995,322555', '316747,316747', '321237,333191', '319715,316747', '8935,322555', '8936,321237', '316747,8995', '8935,327938', '316751,8996', '321237,327938', '8935,8935', '327938,328647', '322555,325071', '322555,325072', '8935,333191', '316747,333191', '8935,316747', '8996,327959', '8935,327940', '316751,8995', '316750,8996', '318151,8935', '316752,8996', '8936,322555', '316752,316747', '8936,327938', '316750,8995', '8996,325072', '316752,8995'

unique pattern: ['8936,8935', '8996,8995', '8995,8995', '8935,8996', '8935,8995', '8995,333191', '8995,327938', '8936,8996', '316751,8936', '8995,327940', '8936,8995', '327940,333191', '327938,327959', '316750,8936', '327959,333191', '316747,8935', '316747,8936', '316751,8935', '8935,321237', '8996,327938', '8995,327959', '325071,333191', '325072,333191', '8996,333191', '327938,333191', '8995,325072', '316750,8935', '8996,327940', '8995,321237', '8995,325071', '321237,322555', '316747,316752', '316747,8996', '8995,322555', '316752,8935', '316747,316747', '321237,333191', '319715,316747', '8935,322555', '316747,8995', '8936,321237', '8935,327938', '316751,8996', '8935,8935', '327938,328647', '8935,333191', '322555,325071', '322555,325072', '321237,327938', '316747,333191', '8935,316747', '8996,327959', '8935,327940', '316751,8995', '316750,8996', '318151,8935', '316752,8996', '8936,322555', '316752,316747', '316750,8995', '8936,327938', '316748,8936', '316748,316747', '316752,8995', '89

unique pattern: ['8936,8935', '8996,8995', '8995,8995', '8935,8996', '8935,8995', '8995,333191', '8995,327938', '8936,8996', '316751,8936', '8995,327940', '8936,8995', '327938,327959', '327940,333191', '316750,8936', '327959,333191', '316747,8935', '316747,8936', '316751,8935', '8935,321237', '8996,327938', '8995,327959', '325071,333191', '325072,333191', '327938,333191', '8996,333191', '8995,325072', '8995,321237', '316750,8935', '8996,327940', '8995,325071', '321237,322555', '316747,316752', '316747,8996', '8995,322555', '316752,8935', '316747,316747', '321237,333191', '319715,316747', '8935,322555', '316747,8995', '8936,321237', '8935,327938', '316751,8996', '8935,8935', '327938,328647', '321237,327938', '8935,333191', '322555,325071', '322555,325072', '316747,333191', '8935,316747', '8996,327959', '8935,327940', '316751,8995', '316750,8996', '318151,8935', '316752,8996', '316752,316747', '8936,322555', '8936,327938', '316750,8995', '316752,8995', '8996,325072', '316748,8936', '3167

unique pattern: ['8936,8935', '8996,8995', '8995,8995', '8935,8996', '8935,8995', '8995,333191', '8995,327938', '8936,8996', '316751,8936', '8995,327940', '8936,8995', '327938,327959', '327940,333191', '316750,8936', '327959,333191', '316747,8935', '316747,8936', '316751,8935', '8935,321237', '8996,327938', '8995,327959', '325071,333191', '325072,333191', '327938,333191', '8995,325072', '8996,333191', '316750,8935', '8995,321237', '8996,327940', '8995,325071', '321237,322555', '316747,8996', '316747,316752', '316752,8935', '8995,322555', '316747,316747', '321237,333191', '319715,316747', '8935,322555', '8936,321237', '316747,8995', '8935,327938', '316751,8996', '8935,8935', '327938,328647', '321237,327938', '322555,325071', '322555,325072', '8935,333191', '316747,333191', '8935,316747', '8996,327959', '8935,327940', '316751,8995', '316750,8996', '318151,8935', '316752,8996', '8936,322555', '316752,316747', '8936,327938', '316750,8995', '8996,325072', '316752,8995', '316748,8936', '3167

In [740]:
sum_bad = 0
for i,student in enumerate(s_bad):
    l2 = list(event_r[event_r.userId==student].objectId)
    len_r = len(l2)
    l1 = recommend(trainingset=l_bad, s_group=s_bad, student=student, path_length=len_r, rl=resourcelist,event=event_r)
    sum_bad += cal_dis(l1,l2)
print(sum_bad/len(s_good))
30.0

student is:  allal11u
unique pattern: ['325068,327938', '8936,8935', '316747,321237', '321237,325068', '316747,316747', '321239,325068', '327938,321237', '327938,327938', '319715,316747', '321237,327938', '8935,316747', '8935,8996', '316747,327938', '321237,321238', '321238,321239', '8935,8995', '316747,316748', '8936,316747', '8995,327938', '316747,321239', '321239,327938', '327938,325068', '316748,321237', '8935,321237', '8996,327938', '8995,321237', '316747,325068', '8935,319715', '8936,8996', '321238,325068', '321237,321239', '8996,321237', '316748,316747', '8936,8995', '319715,321237', '8996,8995', '325068,325069', '8936,319715', '321237,333191', '325069,327938', '8936,321237', '319715,325068', '8995,325068', '8996,325068', '8935,327938', '321237,327960', '8935,8935', '321238,327938', '8995,316747', '8996,316747', '8935,321238', '316748,321238', '316747,321238', '316747,333191', '316748,321239', '327938,333191', '319715,327938', '8995,333191', '8936,327938', '316748,327938', '3250

unique pattern: ['325068,327938', '8936,8935', '316747,321237', '321237,325068', '8935,8996', '327938,321237', '316747,316747', '8935,316747', '327938,327938', '8935,8995', '321239,325068', '321237,327938', '319715,316747', '316747,327938', '321237,321238', '321238,321239', '8995,327938', '8995,321237', '8936,8996', '8996,8995', '8996,327938', '8935,321237', '316747,316748', '316747,321239', '8935,319715', '8936,316747', '321237,327960', '327938,325068', '8996,321237', '8936,8995', '316747,325068', '321239,327938', '316748,316747', '321237,321239', '8935,8935', '319715,321237', '321238,325068', '8995,328647', '8935,327938', '321237,333191', '325068,325069', '8936,319715', '325069,327938', '321237,328647', '8936,321237', '316748,321237', '8995,325068', '8996,325068', '328647,327938', '328647,327960', '319715,325068', '8996,316747', '8935,321238', '316747,321238', '321238,327938', '316747,333191', '327938,327960', '327938,333191', '319715,327938', '8995,316747', '8995,333191', '8996,3286

unique pattern: ['325068,327938', '8936,8935', '316747,321237', '321237,325068', '321239,325068', '316747,316747', '319715,316747', '327938,321237', '327938,327938', '8935,8996', '8935,316747', '321237,327938', '8935,8995', '321237,321238', '321238,321239', '316747,327938', '316747,316748', '8995,327938', '316747,321239', '321239,327938', '8936,8996', '8935,319715', '327938,325068', '8996,327938', '8935,321237', '8936,316747', '316747,325068', '8995,321237', '316748,321237', '8936,8995', '325068,325069', '321238,325068', '321237,321239', '8996,321237', '316748,316747', '319715,321237', '8996,8995', '8936,319715', '321237,333191', '325069,327938', '325069,327960', '8995,325068', '8996,325068', '319715,325068', '8935,8935', '8935,327938', '8936,321237', '321237,327960', '321238,327938', '8995,316747', '8996,316747', '8935,321238', '316748,321238', '316747,321238', '316747,333191', '327938,333191', '319715,327938', '8995,333191', '325068,327960', '316748,321239', '8936,327938', '8935,3250

unique pattern: ['325068,327938', '8936,8935', '321237,325068', '316747,321237', '316747,316747', '327938,327938', '327938,321237', '321239,325068', '8935,316747', '319715,316747', '321237,327938', '316747,327938', '8935,8996', '321237,321238', '321238,321239', '8935,8995', '316747,316748', '321239,327938', '316747,321239', '327938,325068', '8936,316747', '8995,327938', '8935,319715', '8936,8996', '316747,325068', '316748,321237', '8935,321237', '8996,327938', '321238,325068', '321237,321239', '325068,325069', '8936,8995', '8995,321237', '316748,316747', '8996,321237', '319715,321237', '8996,8995', '8936,319715', '8995,325068', '8996,325068', '325069,327938', '325069,327960', '319715,325068', '8936,321237', '8935,327938', '321237,327960', '321238,327938', '321237,333191', '8935,321238', '316748,321238', '316747,321238', '316748,321239', '8935,8935', '319715,327938', '327938,333191', '8995,333191', '325068,327960', '319715,316748', '8996,316747', '316748,327938', '316747,333191', '8935,

In [ ]:
GSS:15.13
NSS:23.4
BSS:25

In [790]:
25/(17.53+3)*13.47-15.13

1.2728251339503185

### Method 3 Markov Chain

In [789]:
16.2/(17.53+3)*13.47

10.629030686799805

In [798]:
sum_good = 0
for i,student in enumerate(s_good):
    l2 = list(event_r[event_r.userId==student].objectId)
    l2 = [resourceIndex[x] for x in l2]
    len_r = len(l2)
    l1 = study_forecast(resource=resourceIndex, states=state,student=student,trainingset=l_good,
                   s_group=s_good, path_length=len_r,event1=event_c,event2=event_r)
    print(l2)
    sum_good += editdistance.eval(str(l1),str(l2))
print(sum_good/len(s_good)/normalisation/(17.53+3)*13.47)

calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 0
Possible states: [0, 4, 13, 5, 15, 33]
Probability of the possible sequence of states: 0.00013088570894231608
[0, 4, 4, 13, 5]
calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 17
Possible states: [17, 32, 14, 5, 21, 15, 25, 26, 20, 21, 15, 25, 26, 20, 0, 14, 36, 24, 13, 20, 0, 4, 0, 4, 24, 14, 20, 0, 4, 24, 13, 20, 0]
Probability of the possible sequence of states: 1.8408758643687668e-10
[17, 32, 14, 36, 24, 13, 20, 0, 16, 15, 25, 26, 20, 21, 22, 5, 6, 4, 0, 14, 20, 0, 4, 24, 14, 5, 21, 15, 15, 15, 5, 26]
calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 18
Possible states: [18, 27, 23, 5, 0, 5, 0, 0]
Probability of the possible sequence of states: 0.009120

In [799]:
sum_normal = 0
for i,student in enumerate(s_normal):
    l2 = list(event_r[event_r.userId==student].objectId)
    l2 = [resourceIndex[x] for x in l2]
    len_r = len(l2)
    l1 = study_forecast(resource=resourceIndex, states=state,student=student,trainingset=l_good,
                   s_group=s_normal, path_length=len_r,event1=event_c,event2=event_r)
    sum_normal += editdistance.eval(str(l1),str(l2))
print(sum_normal/len(s_normal)/normalisation)

calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 4
Possible states: [4, 3, 4, 3, 4, 3, 4, 3, 32, 17, 18, 19, 12, 36, 24, 25, 26, 27, 28, 29, 13, 20, 22, 11, 23, 23, 7, 0, 5, 6, 1, 2, 30, 31, 30, 31, 30, 31]
Probability of the possible sequence of states: 1.5832604518308154e-10
calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 17
Possible states: [17, 17, 27, 1, 4, 30, 3, 4, 30, 31]
Probability of the possible sequence of states: 0.0013510580010612672
calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 17
Possible states: [17, 18, 27, 28, 1, 30, 0, 5, 6, 0, 0, 5, 6, 8, 2, 1]
Probability of the possible sequence of states: 3.19901074986065e-08
calculating the transition matrix from all the records
len M_s: 37
ca

In [796]:
sum_bad = 0
for i,student in enumerate(s_bad):
    l2 = list(event_r[event_r.userId==student].objectId)
    l2 = [resourceIndex[x] for x in l2]
    len_r = len(l2)
    l1 = study_forecast(resource=resourceIndex, states=state,student=student,trainingset=l_normal,
                   s_group=s_bad, path_length=len_r,event1=event_c,event2=event_r)
    sum_bad += editdistance.eval(str(l1),str(l2))
print(sum_bad/len(s_bad)/normalisation)

calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 17
Possible states: [17, 27, 28, 28, 28, 36, 9, 9, 33]
Probability of the possible sequence of states: 3.095169334872022e-07
calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 17
Possible states: [17, 18, 27]
Probability of the possible sequence of states: 0.06284970722186077
calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 17
Possible states: [17, 17, 17, 17, 17, 27, 27, 1, 4, 4]
Probability of the possible sequence of states: 7.223458224829832e-05
calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 27
Possible states: [27, 2, 2, 2, 2, 11, 23]
Probability of the possib

In [770]:
print(sum_bad)

0


# 分析学生学习次数的粗浅Analysis

In [543]:
from pyecharts import options as opts
from pyecharts.charts import Bar
len_good = [len(x) for x in l_good]
len_good.sort(reverse=True)
# d_good = {}
# for i in len_good:
#     if i not in d_good.keys():
#         d_good[i]=1
#     else:
#         d_good[i]+=1

len_bad = [len(x) for x in l_bad]
len_normal = [len(x) for x in l_normal]
len_normal.sort(reverse=True)
len_bad.sort(reverse=True)
def bar_base() -> Bar:
    c = (
        Bar()
        .add_xaxis([i for i in range(len(len_good))])
        .add_yaxis("GSL", len_good[:13])
        .add_yaxis("NSL",len_normal[:13])
        .add_yaxis("BSL",len_bad[6:19])
#         .add_yaxis("", Faker.values())
        
        .set_global_opts(title_opts=opts.TitleOpts(title="Distribution during course part", subtitle="Leaning resourece times during review"))
        .set_series_opts(
            label_opts=opts.LabelOpts(is_show=False),
            toolbox_opts=opts.ToolboxOpts(),
            markline_opts=opts.MarkLineOpts(
                data=[
#                     opts.MarkLineItem(type_="min", name="min"),
#                     opts.MarkLineItem(type_="max", name="max"),
#                     opts.MarkLineItem(type_="average", name="average"),
                ]
            ),
        )
    )
    return c
g = bar_base()
g.render_notebook()

# 最后一个方法

In [802]:
#寻找相似的student
userList = list(set(event.userId))

In [ ]:
    min_dis = 1000
    for j,k in enumerate(s_good):
        lp_normal = str(list(event_c[event_c.userId==e].objectId))
        lp_good = str(list(event_c[event_c.userId==k].objectId))
        distance = editdistance.eval(lp_good, lp_normal)
        if min_dis > distance:
            if len(min_list)>=i+1:
                min_list[i] = [e,k,distance]
            else:
                min_list.append([e,k,distance])

In [826]:
sim = []
for i,u1 in enumerate(userList):
    min_dis = 1000
    for j,u2 in enumerate(userList):
        l1 = str(list(event_c[event_c.userId==u1].objectId))
        l2 = str(list(event_c[event_c.userId==u2].objectId))
        distance = editdistance.eval(l1, l2)
        if min_dis > distance and u1!=u2:
            if len(sim)>=i+1:
                sim[i] = [u1,u2,distance]
                min_dis = distance
            else:
                sim.append([u1,u2,distance])
                min_dis = distance

In [825]:
(list(event_c[event_c.userId=='vuillau51u'].objectId))

[8936,
 8935,
 8995,
 8996,
 327938,
 327938,
 327959,
 327938,
 327960,
 328647,
 327941,
 330711,
 330708,
 8935,
 319715,
 316747,
 316747,
 316748,
 321237,
 325068,
 327938]

In [824]:
(list(event_r[event_r.userId=='vuillau51u'].objectId))

[]

In [504]:
print(len(len_n))
print(sum(len_bad)/19)

19
30.9


#### 以下是课程阶段相似的paires
* ['courtin11u', 'richardo9u', 42],
* ['lauren329u', 'maige1u', 37],
* ['poirot152u', 'dubois173u', 41],
* ['dubois173u', 'poirot152u', 41],
* ['maige1u', 'lauren329u', 37],
* ['oudart4u', 'maige1u', 45],

In [ ]:
print(len(len_n))
print(sum(len_bad)/19)

In [837]:
but = ['courtin11u','lauren329u','poirot152u','oudart4u']
examine = ['richardo9u', 'maige1u','dubois173u','maige1u']
result = []
for i,j in enumerate(but):
    l2 = list(event_r[event_r.userId==examine[i]].objectId)
    l2 = [resourceIndex[x] for x in l2]
    len_r = len(l2)
    l1 = study_forecast(resource=resourceIndex, states=state,student=j,trainingset=l_normal,
                   s_group=s_normal, path_length=len_r,event1=event_c,event2=event_r)
    result.append(editdistance.eval(str(l1),str(l2)))

calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 17
Possible states: [17]
Probability of the possible sequence of states: 1
calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 17
Possible states: [17, 18, 19, 34, 15, 16, 12]
Probability of the possible sequence of states: 6.115811794887884e-05
calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 0
Possible states: [0, 13, 32, 14, 16, 12, 33, 29, 9, 33, 15, 18, 27, 28, 11, 1, 30, 30, 3, 18, 27, 27, 10, 9, 0, 13, 32, 33, 33, 33, 33, 33, 33]
Probability of the possible sequence of states: 1.4508707856392396e-23
calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 17
Possible s

In [972]:
result_pm

[4, 36, 161, 36]

In [978]:
len(list(event_r[event_r.userId=='dubois173u'].objectId))

32

In [975]:
sum([4, 36, 61, 36])/4/4

6.333333333333333

In [973]:
sum([2, 13, 51, 11])/4/4

4.8125

In [969]:
but = ['courtin11u','lauren329u','poirot152u','oudart4u']
examine = ['richardo9u', 'maige1u','dubois173u','maige1u']
result_pm = []
for i,j in enumerate(examine):
    l2 = list(event_r[event_r.userId==examine[i]].objectId)
    l2 = [resourceIndex[x] for x in l2]
    len_r = len(l2)
    l1 = recommend(trainingset=l_normal, s_group=s_normal, student=j, path_length=len_r, rl=resourcelist,event=event_r)
    result_pm.append(editdistance.eval(str(l1),str(l2)))


student is:  richardo9u
unique pattern: ['8936,8935', '8996,8995', '8995,8995', '8935,8996', '8935,8995', '8995,333191', '8995,327938', '8936,8996', '316751,8936', '8995,327940', '8936,8995', '327938,327959', '327940,333191', '316750,8936', '327959,333191', '316747,8936', '316747,8935', '316751,8935', '8935,321237', '8996,327938', '8995,327959', '325071,333191', '325072,333191', '327938,333191', '8995,325072', '8996,333191', '316750,8935', '8995,321237', '8996,327940', '8995,325071', '321237,322555', '316747,8996', '316747,316752', '316752,8935', '8995,322555', '316747,316747', '321237,333191', '8935,322555', '8936,321237', '316747,8995', '8935,327938', '327938,328647', '316751,8996', '8935,8935', '321237,327938', '322555,325071', '322555,325072', '8935,333191', '316747,333191', '8935,316747', '8996,327959', '8935,327940', '316751,8995', '319715,316747', '316750,8996', '318151,8935', '316752,8996', '8936,322555', '316752,316747', '8936,327938', '316750,8995', '8996,325072', '316748,893

In [970]:
result_pm

[4, 36, 161, 36]

In [954]:
but = ['lauren329u','poirot152u','oudart4u']
examine = ['maige1u','dubois173u','maige1u']
result2 = []
for i,j in enumerate(examine):
    l2 = list(event_r[event_r.userId==but[i]].objectId)
    l2 = [resourceIndex[x] for x in l2]
    len_r = len(l2)
    l1 = study_forecast(resource=resourceIndex, states=state,student=j,trainingset=l_normal,
                   s_group=s_normal, path_length=len_r,event1=event_c,event2=event_r)
    result2.append(editdistance.eval(str(l1),str(l2)))

calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 27
Possible states: [27, 2, 2]
Probability of the possible sequence of states: 0.5438437001594896
calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 18
Possible states: [18, 27, 18, 18, 17, 27, 27, 28, 11, 23, 2, 30, 1, 1, 8, 22, 21, 22, 16, 26, 24]
Probability of the possible sequence of states: 3.3063080883357115e-18
calculating the transition matrix from all the records
len M_s: 37
calculating the transition matrix from all the records
len M 37
Start state: 27
Possible states: [27, 2, 11, 23, 20, 21, 5, 5, 8, 30, 14, 15, 16, 36, 36, 29, 10, 9]
Probability of the possible sequence of states: 1.3226876542899298e-15


In [959]:
result_pm

[]

In [955]:
result2

[5, 26, 29]

In [971]:
13/18

0.7222222222222222

In [843]:
#'courtin11u'的推荐路径和 'richardo9u'的学习路径完全一致 
#'dubois173u'的推荐路径和 'poirot152u'的学习路径完全一致
Studengexam = list(exam.userId)

,Unnamed: 0,userId,score,action,Timestamp,objectId
129,129,wolter6u,100.000000,FinalExam,1542835584754,11209
7,7,bailly88u,94.444444,FinalExam,1542835584754,11209
79,79,marais8u,91.666667,FinalExam,1542835584754,11209
43,43,gonand2u,90.277778,FinalExam,1542835584754,11209
42,42,gonand2u,90.277778,FinalExam,1542835584754,11209
...,...,...,...,...,...,...
83,83,massulah1u,2.777778,FinalExam,1542835584754,11209
3,3,amineelh3u,2.777778,FinalExam,1542835584754,11209
131,131,wu61u,2.777778,FinalExam,1542835584754,11209
6,6,baamr4u,1.388889,FinalExam,1542835584754,11209


In [848]:
Studengexam = list(exam.userId)
print(Studengexam.index('poirot152u'))

12


In [849]:
event.head(2)

,userId,objectId,action,Timestamp
3078,debano2u,316747,Viewed,1537195506755
3079,debano2u,316748,Viewed,1537195548755


In [ ]:
MC:
courtin11u quiz 5.8 exam 11.75 rank 42
richardo9u quiz 10 exam 13.5 rank 22 进步两分 

dubois173u quiz 8.5 exam 3.75 rank 103
poirot152u quiz 10 exam 15  rank 12

PM:
courtin11u quiz 5.8 exam 11.75 rank 42
richardo9u quiz 10 exam 13.5 rank 22 进步两分 


In [851]:
22+91/2

67.5

### 作图

In [1046]:
c_good = []
c_bad = []
c_normal = []
for user in s_good:
    l_path = list(event_r[event_r.userId==user].objectId)
    c_good.append(l_path)
for user in s_normal:
    l_path = list(event_r[event_r.userId==user].objectId)
    c_normal.append(l_path)
for user in s_bad:
    l_path = list(event_r[event_r.userId==user].objectId)
    c_bad.append(l_path)

len_bad = [len(x) for x in c_bad]
len_normal = [len(x) for x in c_normal]
len_good = [len(x) for x in c_good]
len_normal.sort(reverse=True)
len_bad.sort(reverse=True)
len_good.sort(reverse=True)
def bar_base() -> Bar:
    c = (
        Bar()
        .add_xaxis([i+1 for i in range(15)])
        .add_yaxis("GSL", len_good)
        .add_yaxis("NSL",len_normal)
        .add_yaxis("BSL",len_bad[6:19])
#         .add_yaxis("", Faker.values())
        
        .set_global_opts(title_opts=opts.TitleOpts(title="Distribution during review part", subtitle="Leaning resourece times during review"))
        .set_series_opts(
            label_opts=opts.LabelOpts(is_show=False),
            toolbox_opts=opts.ToolboxOpts(),
            markline_opts=opts.MarkLineOpts(
                data=[
#                     opts.MarkLineItem(type_="min", name="min"),
#                     opts.MarkLineItem(type_="max", name="max"),
                    #opts.MarkLineItem(type_="average", name="average"),
                ]
            ),
        )
    )
    return c
g = bar_base()
g.render_notebook()

In [1048]:
for i,e in enumerate(len_good):
    print((i+1,e))

(1, 94)
(2, 32)
(3, 20)
(4, 17)
(5, 17)
(6, 16)
(7, 13)
(8, 13)
(9, 11)
(10, 9)
(11, 7)
(12, 5)
(13, 5)
(14, 2)
(15, 2)


In [1049]:
for i,e in enumerate(len_normal):
    print((i+1,e))

(1, 37)
(2, 19)
(3, 19)
(4, 19)
(5, 17)
(6, 15)
(7, 12)
(8, 12)
(9, 9)
(10, 7)
(11, 4)
(12, 4)
(13, 1)


In [1051]:
for i,e in enumerate(len_bad[4:19]):
    print((i+1,e))

(1, 15)
(2, 11)
(3, 11)
(4, 10)
(5, 10)
(6, 9)
(7, 8)
(8, 8)
(9, 6)
(10, 6)
(11, 2)
(12, 1)
(13, 1)
(14, 1)
(15, 1)


In [1056]:
lc_good = []
lc_bad = []
lc_normal = []
for user in s_good:
    l_path = list(event_c[event_c.userId==user].objectId)
    lc_good.append(l_path)
for user in s_normal:
    l_path = list(event_c[event_c.userId==user].objectId)
    lc_normal.append(l_path)
for user in s_bad:
    l_path = list(event_c[event_c.userId==user].objectId)
    lc_bad.append(l_path)


In [1059]:
lc_g = [len(x) for x in lc_good]
lc_n = [len(x) for x in lc_normal]
lc_b = [len(x) for x in lc_bad]
lc_g.sort(reverse=True)
lc_n.sort(reverse=True)
lc_b.sort(reverse=True)

for i,e in enumerate(lc_g):
    print((i+1,e))

for i,e in enumerate(lc_n):
    print((i+1,e))
for i,e in enumerate(lc_b[4:]):
    print((i+1,e))

(1, 120)
(2, 108)
(3, 94)
(4, 82)
(5, 73)
(6, 57)
(7, 54)
(8, 45)
(9, 44)
(10, 32)
(11, 30)
(12, 28)
(13, 20)
(14, 19)
(15, 13)
(1, 145)
(2, 101)
(3, 62)
(4, 55)
(5, 48)
(6, 46)
(7, 33)
(8, 33)
(9, 31)
(10, 31)
(11, 25)
(12, 21)
(13, 17)
(1, 59)
(2, 59)
(3, 58)
(4, 58)
(5, 45)
(6, 45)
(7, 45)
(8, 45)
(9, 42)
(10, 34)
(11, 34)
(12, 25)
(13, 10)
(14, 10)
(15, 7)


In [528]:
print(sum([len(x) for x in c_bad])/19)
print(sum([len(x) for x in c_normal])/13)
print(sum([len(x) for x in c_good])/15)

13.473684210526315
13.461538461538462
17.533333333333335


In [501]:
6.4*0.4+5.8*0.3+8.76*0.3

6.928000000000001

# 一个判断推荐路径长度的表格

In [933]:
studentList =s_good[:10]+s_bad[:10]+s_normal[:10]
#print(len(studentList))
dict_r ={5:0,7:0,8:0,9:0,10:0,11:0,12:0,13:0,15:0,17:0,20:0,}
for key,v in dict_r.items():
    sumbad = 0
    for i,student in enumerate(studentList):
        l2 = list(event_r[event_r.userId==student].objectId)
        l2 = [resourceIndex[x] for x in l2]
        len_r = len(l2)
        sumbad += abs(len_r-key)
    dict_r[key] = sumbad/len(studentList) -5
print(dict_r)

{5: 7.0, 7: 6.066666666666666, 8: 5.800000000000001, 9: 5.666666666666666, 10: 5.7333333333333325, 11: 5.866666666666667, 12: 6.066666666666666, 13: 6.333333333333334, 15: 7.0, 17: 7.800000000000001, 20: 9.333333333333334}


In [934]:
#s_good
#s_bad
#s_normal
#print(len(studentList))
dict_good ={5:0,7:0,8:0,9:0,10:0,11:0,12:0,13:0,15:0,17:0,20:0,}

for key,v in dict_good.items():
    sumbad = 0
    for i,student in enumerate(s_good):
        l2 = list(event_r[event_r.userId==student].objectId)
        l2 = [resourceIndex[x] for x in l2]
        len_r = len(l2)
        sumbad += abs(len_r-key)
    dict_good[key] = sumbad/len(s_good)-5
print(dict_good)

dict_normal ={5:0,7:0,8:0,9:0,10:0,11:0,12:0,13:0,15:0,17:0,20:0,}

for key,v in dict_normal.items():
    sumbad = 0
    for i,student in enumerate(s_normal):
        l2 = list(event_r[event_r.userId==student].objectId)
        l2 = [resourceIndex[x] for x in l2]
        len_r = len(l2)
        sumbad += abs(len_r-key)
    dict_normal[key] = sumbad/len(s_normal)-5
print(dict_normal)

dict_bad ={5:0,7:0,8:0,9:0,10:0,11:0,12:0,13:0,15:0,17:0,20:0,}

for key,v in dict_bad.items():
    sumbad = 0
    for i,student in enumerate(s_bad):
        l2 = list(event_r[event_r.userId==student].objectId)
        l2 = [resourceIndex[x] for x in l2]
        len_r = len(l2)
        sumbad += abs(len_r-key)
    dict_bad[key] = sumbad/len(s_bad)-5
print(dict_bad)

{5: 8.333333333333334, 7: 7.4, 8: 7.066666666666666, 9: 6.7333333333333325, 10: 6.533333333333333, 11: 6.333333333333334, 12: 6.2666666666666675, 13: 6.199999999999999, 15: 6.6, 17: 7.133333333333333, 20: 8.933333333333334}
{5: 4.384615384615385, 7: 3.3076923076923084, 8: 2.9230769230769234, 9: 2.5384615384615383, 10: 2.3076923076923075, 11: 2.0769230769230766, 12: 1.8461538461538458, 13: 1.9230769230769234, 15: 2.0769230769230766, 17: 2.5384615384615383, 20: 4.153846153846153}
{5: 5.473684210526315, 7: 4.7368421052631575, 8: 4.473684210526315, 9: 4.421052631578947, 10: 4.473684210526315, 11: 4.7368421052631575, 12: 5.210526315789474, 13: 5.684210526315789, 15: 6.631578947368421, 17: 7.789473684210526, 20: 9.526315789473685}


In [935]:
dict_r.values()

dict_values([7.0, 6.066666666666666, 5.800000000000001, 5.666666666666666, 5.7333333333333325, 5.866666666666667, 6.066666666666666, 6.333333333333334, 7.0, 7.800000000000001, 9.333333333333334])

In [926]:
L = []
dict_r
for k, v in dict_r.items():
    L.append((k,round(v,1)))

In [1063]:
L

[(5, 7.0),
 (7, 6.1),
 (8, 5.8),
 (9, 5.7),
 (10, 5.7),
 (11, 5.9),
 (12, 6.1),
 (13, 6.3),
 (15, 7.0),
 (17, 7.8),
 (20, 9.3)]

In [928]:
L_bad = []
for k, v in dict_bad.items():
    L_bad.append((k,round(v,1)))

In [930]:
L_normal = []
for k, v in dict_normal.items():
    L_normal.append((k,round(v,1)))

In [931]:
L_good = []
for k, v in dict_good.items():
    L_good.append((k,round(v,1)))

In [936]:
print(L_good)
print(L_normal,L_bad)

[(5, 8.3), (7, 7.4), (8, 7.1), (9, 6.7), (10, 6.5), (11, 6.3), (12, 6.3), (13, 6.2), (15, 6.6), (17, 7.1), (20, 8.9)]
[(5, 4.4), (7, 3.3), (8, 2.9), (9, 2.5), (10, 2.3), (11, 2.1), (12, 1.8), (13, 1.9), (15, 2.1), (17, 2.5), (20, 4.2)] [(5, 5.5), (7, 4.7), (8, 4.5), (9, 4.4), (10, 4.5), (11, 4.7), (12, 5.2), (13, 5.7), (15, 6.6), (17, 7.8), (20, 9.5)]


In [945]:
(event_c[event_c.userId=='lauren329u'].objectId)

6541    316751
58        8936
6543      8935
6546      8995
111       8996
6548      8995
6551    318151
Name: objectId, dtype: int64

In [944]:
(event_c[event_c.userId=='maige1u'].objectId)

6734      8935
53        8936
6737    316747
6739      8935
6742    321240
148       8996
6743      8995
6746    321240
6749    327941
6751    333191
Name: objectId, dtype: int64

In [949]:
len(event_c[event_c.userId=='courtin11u'].objectId)

17

In [948]:
len(event_c[event_c.userId=='richardo9u'].objectId)

18